# Ejercicio Clustering. Detección de Anomalías

Uno de los usos de los algoritmos de clustering es la Detección de Anomalías, esto es, la detección de observaciones anómalas, aquellas que no siguen un comportamiento normal. Si el objetivo del clustering es encontrar grupos de elementos similares, aquellos elementos que no son similares a ningún grupo se pueden considerar como elementos anómalos.

Para este ejercicio vamos a usar un [Dataset de transacciones de tarjetas de crédito](https://www.kaggle.com/arjunbhasin2013/ccdata), donde cada observacion es un cliente distinto.

Nuestro objetivo es implementar un modelo que agrupa las transacciones apropiadamente y encontrar los potenciales outliers, es decir, aquellas transacciones que son sospechosas de ser un fraude o un error. Para resolver este ejercicio correctamente hay que investigar, en vez de simplemente seguir a rajatabla lo enseñado en el curso.

**Pistas:**

- Hemos explicado un algoritmo de clustering que no solo asigna elementos a clusters válidos, sino que también clasifica elementos como valores extremos (outliers). 

- Para la búsqueda de hiperparámetros, un buen sitio para mirar es [ParameterSampler](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterSampler.html).

In [53]:
import pandas as pd
df = pd.read_csv("data/CC GENERAL.csv")

In [54]:
df.shape

(8950, 18)

In [55]:
df.dtypes

CUST_ID                              object
BALANCE                             float64
BALANCE_FREQUENCY                   float64
PURCHASES                           float64
ONEOFF_PURCHASES                    float64
INSTALLMENTS_PURCHASES              float64
CASH_ADVANCE                        float64
PURCHASES_FREQUENCY                 float64
ONEOFF_PURCHASES_FREQUENCY          float64
PURCHASES_INSTALLMENTS_FREQUENCY    float64
CASH_ADVANCE_FREQUENCY              float64
CASH_ADVANCE_TRX                      int64
PURCHASES_TRX                         int64
CREDIT_LIMIT                        float64
PAYMENTS                            float64
MINIMUM_PAYMENTS                    float64
PRC_FULL_PAYMENT                    float64
TENURE                                int64
dtype: object

In [56]:
customer_ids = df.CUST_ID
df = df.drop(columns="CUST_ID")

In [57]:
df.columns[df.isnull().any()]

Index(['CREDIT_LIMIT', 'MINIMUM_PAYMENTS'], dtype='object')

In [58]:
df[df.isnull().any(axis=1)].shape

(314, 17)

Imputamos a 0 los valores nulos, así podemos ver si los elementos anómalos son aquellos que tienen estas columnas a 0

In [59]:
df = df.fillna(0)

Hemos aprendido el algoritmo DBSCAN, lo bueno de usar este algoritmo para detección de anomalías es que no asigna un cluster a todos los puntos, sino aquellos puntos que están más separados del resto se etiquetan automáticamente como valores extremos.

In [60]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

Al usar un algoritmo de clustering basado en densidad tenemos que estandarizar los datos.

In [61]:
df_normalizado = pd.DataFrame(StandardScaler().fit_transform(df))

In [62]:
clusterer = DBSCAN()
cluster_labels = clusterer.fit_predict(df_normalizado)
pd.Series(cluster_labels).value_counts()

-1     6627
 0     1948
 10      60
 2       34
 15      30
 7       23
 14      23
 8       14
 6       13
 3       11
 29      10
 21       9
 5        9
 9        8
 12       8
 1        8
 26       8
 17       7
 11       7
 27       7
 19       7
 4        6
 13       6
 23       6
 28       5
 30       5
 16       5
 24       5
 22       5
 25       5
 20       5
 18       5
 35       5
 31       5
 32       4
 34       4
 33       3
dtype: int64

Vemos que por defecto DBSCAN produce demasiados outliers (más de 7000 elementos anómalos del total de 9000 transacciones. Podemos usar el coeficiente de silueta (`silhouette_score`) para ver como separa el algoritmo a los buenos clientes de los (potencialmente) malos.

In [63]:
from sklearn.metrics import silhouette_score

In [64]:
silhouette_score(df_normalizado, cluster_labels)

-0.46596190778573116

Un coeficiente de silueta negativo es bastante malo. Podemos hacer una búsqueda aleatoria para optimizar dicho resultado.

In [65]:
DBSCAN().get_params()

{'algorithm': 'auto',
 'eps': 0.5,
 'leaf_size': 30,
 'metric': 'euclidean',
 'metric_params': None,
 'min_samples': 5,
 'n_jobs': 1,
 'p': None}

In [66]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform 


distribucion_parametros = {
    "eps": uniform(0,5),
    "min_samples": sp_randint(2, 20),
    "p": sp_randint(1, 3),
}

In [67]:
distribucion_parametros

{'eps': <scipy.stats._distn_infrastructure.rv_frozen at 0x21816e979b0>,
 'min_samples': <scipy.stats._distn_infrastructure.rv_frozen at 0x2181586efd0>,
 'p': <scipy.stats._distn_infrastructure.rv_frozen at 0x218155fbf98>}

Un problema que hay con el algoritmo HDBSCAN (o DBSCAN) es que no tiene el método `predict`, por lo tanto no podemos usar el método de busqueda aleatoria (RandomSearchCV) de scikit-learn.

Sin embargo, podemos desarrollar nuestro propio método de búsqueda con  [`ParameterSampler`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterSampler.html), que es lo que usa scikit-learn para tomar muestras del diccionario de búsqueda de hiperparámetros.

**Este paso tarda tiempo en ejecutarse!**

In [21]:
import numpy as np
from sklearn.model_selection import ParameterSampler

n_muestras = 30 # probamos 20 combinaciones de hiperparámetros
n_iteraciones = 3 #para validar, vamos a entrenar para cada selección de hiperparámetros en 3 muestras distintas
pct_muestra = 0.7 # usamos el 70% de los datos para entrenar el modelo en cada iteracion
resultados_busqueda = []
lista_parametros = list(ParameterSampler(distribucion_parametros, n_iter=n_muestras))

for param in lista_parametros:
    for iteration in range(n_iteraciones):
        param_resultados = []
        muestra = df_normalizado.sample(frac=pct_muestra)
        etiquetas_clusters = DBSCAN(n_jobs=-1, **param).fit_predict(muestra)
        try:
            param_resultados.append(silhouette_score(muestra, etiquetas_clusters))
        except ValueError: # a veces silhouette_score falla en los casos en los que solo hay 1 cluster
            pass
    puntuacion_media = np.mean(param_resultados)
    resultados_busqueda.append([puntuacion_media, param])

In [22]:
sorted(resultados_busqueda, key=lambda x: x[0], reverse=True)[:5]

[[0.75007119325561733, {'eps': 4.9856603649238247, 'min_samples': 6, 'p': 1}],
 [0.74150194018465343, {'eps': 4.9012580110900821, 'min_samples': 7, 'p': 2}],
 [0.73747817850020003, {'eps': 4.9832059188723079, 'min_samples': 17, 'p': 2}],
 [0.72164854282937896, {'eps': 4.5196579162041299, 'min_samples': 9, 'p': 2}],
 [0.71549187189599339, {'eps': 4.4048553408796298, 'min_samples': 9, 'p': 1}]]

In [76]:
mejores_params = {'eps': 4.9856603649238247, 'min_samples':6, 'p': 1}

clusterer = DBSCAN(n_jobs=-1, **mejores_params)

etiquetas_cluster = clusterer.fit_predict(df_normalizado)

In [77]:
pd.Series(etiquetas_cluster).value_counts()

 0    8890
-1      60
dtype: int64

Vemos que hay 60 anomalias potenciales.

In [78]:
def resumen_cluster(cluster_id):
    cluster = df[etiquetas_cluster==cluster_id]
    resumen_cluster = cluster.mean().to_dict()
    resumen_cluster["cluster_id"] = cluster_id
    return resumen_cluster

def comparar_clusters(*cluster_ids):
    resumenes = []
    for cluster_id in cluster_ids:
        resumenes.append(resumen_cluster(cluster_id))
    return pd.DataFrame(resumenes).set_index("cluster_id").T

In [80]:
comparar_clusters(0,-1)

cluster_id,0,-1
BALANCE,1533.399657,6168.779310
BALANCE_FREQUENCY,0.876730,0.957424
CASH_ADVANCE,945.303656,5952.449284
CASH_ADVANCE_FREQUENCY,0.134145,0.283207
CASH_ADVANCE_TRX,3.136558,19.883333
CREDIT_LIMIT,4435.728699,13120.000000
INSTALLMENTS_PURCHASES,381.355524,4813.413500
MINIMUM_PAYMENTS,777.355947,9224.292246
ONEOFF_PURCHASES,531.828127,9572.707000
ONEOFF_PURCHASES_FREQUENCY,0.200307,0.521089
